# BÀI 2: PHÂN TÍCH DỮ LIỆU VÀ TRỰC QUAN HÓA (Pandas/NumPy/Matplotlib/Seaborn)

**Mục tiêu buổi học**
- Ôn nhanh lý thuyết cốt lõi về phân tích dữ liệu (EDA) và trực quan hóa.
- Thực hành demo thao tác dữ liệu với Pandas/NumPy.
- Làm **một bài phân tích dữ liệu hoàn chỉnh** trên dataset online đủ lớn (≈ 48k dòng).

**Dataset dùng cho bài thực hành lớp:** NYC Airbnb 2019 (AB_NYC_2019.csv)  
Nguồn: một bản public trên GitHub (raw CSV).  
URL: `https://raw.githubusercontent.com/ashwinjohn3/New-York-City-Airbnb-Open-Data/master/AB_NYC_2019.csv`


---
## 0) Chuẩn bị môi trường
Khuyến nghị chạy trên Google Colab hoặc JupyterLab local.

Cài thêm (nếu thiếu):
- `pandas`, `numpy`, `matplotlib`, `seaborn`


In [ ]:
# Nếu chạy trên Colab và chưa có seaborn:
    # !pip -q install pandas numpy matplotlib seaborn

---
## 1) ÔN TẬP LÝ THUYẾT (tóm tắt)

### 1.1 Khung quy trình phân tích dữ liệu (EDA)
**EDA (Exploratory Data Analysis)** là giai đoạn khám phá dữ liệu trước khi mô hình hóa.
1. Xác định câu hỏi/giả thuyết
2. Nạp dữ liệu & xem cấu trúc
3. Làm sạch: thiếu dữ liệu, trùng lặp, ngoại lệ
4. Biến đổi/chuẩn hóa: kiểu dữ liệu, tạo biến mới
5. Thống kê mô tả: min/max/mean/median, phân phối
6. Trực quan hóa: phân phối, so sánh nhóm, tương quan
7. Kết luận: insight + đề xuất bước tiếp theo

### 1.2 Các lỗi thường gặp
- Dữ liệu thiếu (missing) nhưng xử lý “mù”: drop quá nhiều gây mất thông tin.
- Ngoại lệ (outlier) làm méo trung bình: cân nhắc median/IQR.
- Trực quan hóa sai loại biểu đồ: ví dụ dùng pie cho dữ liệu nhiều nhóm.
- Quên kiểm soát đơn vị: tiền tệ, ngày tháng, tỷ lệ.

### 1.3 Checklist nhanh (đúng chuẩn làm bài)
- Dataset có bao nhiêu dòng/cột?
- Cột nào numeric/categorical/datetime?
- Missing ở đâu? tỷ lệ bao nhiêu?
- Phân phối biến mục tiêu (vd: price) ra sao?
- So sánh theo nhóm (vd: borough/room_type) có gì khác?
- Tương quan (correlation) gợi ý điều gì?

---
## 2) DEMO MINH HỌA NHANH (Pandas căn bản)


In [ ]:
#%pip install pandas numpy

import pandas as pd
import numpy as np

demo = pd.DataFrame({
    "nhom": ["A","A","B","B","C"],
    "diem": [8, 9, 6, np.nan, 7],
    "gio_hoc": [2, 3, 1, 4, 2]
})

demo

In [ ]:
# 2.1 Xem nhanh cấu trúc
demo.info()

In [ ]:
# 2.2 Missing values
demo.isna().sum()

In [ ]:
# 2.3 Điền missing bằng trung bình
demo_filled = demo.copy()
demo_filled["diem"] = demo_filled["diem"].fillna(demo_filled["diem"].mean())
demo_filled

In [ ]:
# 2.4 Groupby: điểm trung bình theo nhóm
demo_filled.groupby("nhom")["diem"].mean().sort_values(ascending=False)

---
## 3) BÀI THỰC HÀNH LỚP: PHÂN TÍCH DỮ LIỆU HOÀN CHỈNH (NYC Airbnb 2019)

### 3.1 Câu hỏi phân tích gợi ý (bạn có thể chọn 3–5 câu)
1. Giá thuê (price) phân phối thế nào? Có outlier không?
2. Giá khác nhau ra sao theo **neighbourhood_group** (borough) và **room_type**?
3. Khu vực nào nhiều listing nhất? Host nào hoạt động nhiều nhất?
4. Availability (availability_365) có liên quan gì đến giá hoặc số review?
5. Có thể rút ra 3 insight thực tế cho người muốn tối ưu giá/đầu tư?

> Lưu ý: Dataset này thường có outlier giá rất cao. Bạn cần xử lý/giới hạn để biểu đồ không bị “vỡ”.

---
### 3.2 Nạp dữ liệu từ nguồn online


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/ashwinjohn3/New-York-City-Airbnb-Open-Data/master/AB_NYC_2019.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

---
### 3.3 Kiểm tra chất lượng dữ liệu


In [ ]:
# Missing theo cột
missing = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
missing.head(15)

In [ ]:
# Dòng trùng lặp
df.duplicated().sum()

In [ ]:
# Thống kê mô tả (chỉ numeric)
df.describe().T

---
### 3.4 Làm sạch + chuẩn hóa cơ bản

Ta sẽ:
- Loại bỏ dòng thiếu trường quan trọng (ví dụ: `name`, `host_name` có thể cho phép thiếu; nhưng price, room_type, neighbourhood_group nên có).
- Xử lý outlier giá: tạo cột `price_clipped` giới hạn theo percentile.
- Đảm bảo kiểu dữ liệu đúng.

> Trong bài dạy, mục tiêu là minh họa quy trình. Tùy lớp, bạn có thể yêu cầu SV thử nhiều chiến lược làm sạch khác nhau và so sánh.


In [ ]:
df_clean = df.copy()

# (1) Loại bỏ các giá không hợp lệ (<=0)
df_clean = df_clean[df_clean["price"] > 0].copy()

# (2) Cắt outlier theo percentile (1% - 99%) để trực quan hóa đẹp
low, high = df_clean["price"].quantile([0.01, 0.99])
df_clean["price_clipped"] = df_clean["price"].clip(lower=low, upper=high)

# (3) Điền thiếu ở name/host_name bằng chuỗi
for col in ["name", "host_name", "last_review"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna("Unknown")

df_clean.shape, (low, high)

---
### 3.5 Trực quan hóa (Matplotlib/Seaborn)

Chúng ta sẽ vẽ:
- Histogram phân phối giá (đã clip outlier)
- Boxplot giá theo borough
- Barplot số lượng listing theo borough
- Heatmap tương quan numeric


In [ ]:
%pip install matplotlib seaborn

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Histogram phân phối giá (đã clip)
plt.figure(figsize=(10,4))
plt.hist(df_clean["price_clipped"], bins=60)
plt.title("Phân phối giá thuê (price_clipped)")
plt.xlabel("Giá (USD)")
plt.ylabel("Số lượng listing")
plt.show()

In [ ]:
# Số lượng listing theo borough
counts_borough = df_clean["neighbourhood_group"].value_counts()

plt.figure(figsize=(8,4))
plt.bar(counts_borough.index, counts_borough.values)
plt.title("Số lượng listing theo borough (neighbourhood_group)")
plt.xlabel("Borough")
plt.ylabel("Số lượng")
plt.xticks(rotation=30, ha="right")
plt.show()

counts_borough

In [ ]:
# Boxplot giá theo borough (dùng price_clipped)
plt.figure(figsize=(10,4))
sns.boxplot(data=df_clean, x="neighbourhood_group", y="price_clipped")
plt.title("Giá theo borough (đã clip outlier)")
plt.xlabel("Borough")
plt.ylabel("Giá (USD)")
plt.xticks(rotation=30, ha="right")
plt.show()

In [ ]:
# Giá theo room_type (so sánh median/mean)
price_by_room = df_clean.groupby("room_type")["price"].agg(["count","mean","median"]).sort_values("median", ascending=False)
price_by_room

In [ ]:
# Boxplot giá theo room_type
plt.figure(figsize=(10,4))
sns.boxplot(data=df_clean, x="room_type", y="price_clipped")
plt.title("Giá theo loại phòng (room_type) - đã clip outlier")
plt.xlabel("Room type")
plt.ylabel("Giá (USD)")
plt.xticks(rotation=15, ha="right")
plt.show()

In [ ]:
import numpy as np

# Heatmap tương quan numeric
num_cols = df_clean.select_dtypes(include=[np.number]).columns
corr = df_clean[num_cols].corr(numeric_only=True)

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=False)
plt.title("Tương quan giữa các biến số")
plt.show()

corr

---
### 3.6 Một số insight mẫu (để SV học cách viết kết luận)
Gợi ý cách viết kết luận **đúng chuẩn báo cáo**:
- Không chỉ mô tả biểu đồ, mà phải trả lời câu hỏi phân tích.
- Nêu số liệu/quan sát cụ thể.
- Đề xuất hành động hoặc giả thuyết tiếp theo.

**Ví dụ (SV cần tự hoàn thiện theo kết quả chạy máy):**
1. Borough có nhiều listing nhất là ... → gợi ý thị trường tập trung ở ...
2. Room type có median price cao nhất là ... → người đầu tư nên cân nhắc ...
3. Correlation giữa reviews và availability thường không mạnh → cần thêm biến khác (location quality, host rating...) nếu muốn dự đoán.

---
## 4) BÀI TẬP LỚP (YÊU CẦU NỘP)
### Bài tập: “Báo cáo EDA hoàn chỉnh”
SV làm trên chính dataset này (hoặc TA cho dataset InsideAirbnb phiên bản mới), nộp 1 file `.ipynb` gồm:

1. **Giới thiệu dataset + mục tiêu**
2. **Làm sạch dữ liệu** (nêu rõ quyết định xử lý missing/outlier)
3. **EDA + trực quan hóa**: ít nhất 6 biểu đồ, trong đó có:
   - 1 biểu đồ phân phối (hist/kde)
   - 1 biểu đồ so sánh theo nhóm (box/violin)
   - 1 biểu đồ top-k (bar)
   - 1 heatmap/correlation
4. **Insight & kết luận**: ít nhất 5 gạch đầu dòng, có số liệu minh họa
5. (Tuỳ chọn) **Một mô hình dự đoán đơn giản** (LinearRegression/KNN) và đánh giá

### Thang điểm gợi ý (10 điểm)
- Đúng quy trình, notebook sạch đẹp: 2đ
- Làm sạch hợp lý + giải thích: 2đ
- Biểu đồ đúng loại + rõ ràng: 3đ
- Insight tốt, có số liệu: 3đ


---
## 5) Flowchart (vẽ dạng văn bản để SV nhớ quy trình)

```text
[Đặt câu hỏi] 
      |
      v
[Nạp dữ liệu] -> [Khám phá cấu trúc (shape/info/head)]
      |
      v
[Làm sạch] -> missing -> outlier -> kiểu dữ liệu
      |
      v
[EDA] -> thống kê mô tả -> groupby/pivot
      |
      v
[Trực quan hóa] -> phân phối -> so sánh nhóm -> tương quan
      |
      v
[Kết luận] -> insight -> đề xuất bước tiếp theo / mô hình
```
